In [ ]:
import numpy as np
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

### Hard Margin SVM with `cvxopt` Library

In this section, we implement the Hard Margin SVM using the `cvxopt` library for convex optimization. We also incorporate multiple kernel functions to explore different decision boundaries. To achieve this, we optimize the following dual form of the SVM:

$$
\max_{\alpha} \; W(\alpha) = \sum_{i=1}^{m} \alpha_i - \frac{1}{2} \sum_{i,j=1}^{m} y^{(i)} y^{(j)} \alpha_i \alpha_j \langle x^{(i)}, x^{(j)} \rangle
$$

subject to:
$$
\alpha_i \geq 0, \quad i = 1, \dots, m
$$
and
$$
\sum_{i=1}^{m} \alpha_i y^{(i)} = 0
$$

This optimization problem is quadratic and subject to linear constraints.

### Standard Form for `cvxopt.solvers.qp`

The `cvxopt.solvers.qp` function solves problems in the following standard form:

$$
\begin{aligned}
&\min \frac{1}{2} x^T P x + q^T x \\
\text{subject to:} \quad & G x \leq h \\
& A x = b
\end{aligned}
$$


$$
\begin{aligned}
&\text{where:} \\
&x \text{ is the vector of variables we are solving for (in our case, the vector of Lagrange multipliers } \alpha\text{)} \\
&P \text{ is a matrix representing the quadratic coefficients of the objective function} \\
&q \text{ is a vector representing the linear coefficients of the objective function} \\
&G \text{ and } h \text{ represent inequality constraints} \\
&A \text{ and } b \text{ represent equality constraints}
\end{aligned}
$$


### Mapping the SVM Dual Problem to the QP Form

1. **Variables**:
   - Here, $$ x = \alpha $$, the vector of Lagrange multipliers.

2. **Objective Function**:
   - The dual objective function can be written as:
     $$
     W(\alpha) = \sum_{i=1}^{m} \alpha_i - \frac{1}{2} \sum_{i,j=1}^{m} y^{(i)} y^{(j)} \alpha_i \alpha_j K_{ij}
     $$
     where $$ K_{ij} = \langle x^{(i)}, x^{(j)} \rangle $$ is the Gram matrix (or kernel matrix).
   
   - Since `cvxopt` expects a minimization problem, we need to minimize $$ -W(\alpha) $$, which becomes:
     $$
     \min \; -\sum_{i=1}^{m} \alpha_i + \frac{1}{2} \sum_{i,j=1}^{m} y^{(i)} y^{(j)} \alpha_i \alpha_j K_{ij}
     $$

   - This gives:
     $$
     \begin{align}
     &\text{Matrix } P: & P_{ij} &= y^{(i)} y^{(j)} K_{ij} \\
     &\text{Vector } q: & q_i &= -1 \quad \text{(since the linear term is } -\sum_{i=1}^{m} \alpha_i \text{)}
     \end{align}
     $$

3. **Constraints**:
   $$
   \begin{align}
   &\text{Inequality Constraint } (\alpha_i \geq 0): & G &= -I \quad (\text{negative identity matrix}), \quad h = 0 \\
   &\text{Equality Constraint } \left(\sum_{i=1}^{m} \alpha_i y^{(i)} = 0\right): & A &= y^T \quad \text{(labels vector)}, \quad b = 0
   \end{align}
   $$


#### Kernel Functions

1. **Linear Kernel**  
   $$ K(x_1, x_2) = x_1 \cdot x_2 $$

2. **Polynomial Kernel**  
   $$ K(x_1, x_2) = (x_1 \cdot x_2 + \text{c})^{\text{degree}} $$


In [ ]:
def compute_kernel_matrix(x, y, kwargs):

    kernel_type = kwargs['kernel_type']

    result = None

    ######## YOUR CODE HERE ########
 
    
    ################################
    
    return result

In [ ]:
def find_alpha(X, y, K):

    n_samples, n_features = X.shape
    P_numpy, q_numpy, G_numpy, h_numpy, A_numpy, b_numpy = None, None, None, None, None, None

    # Convert inputs to cvxopt format
    ######### YOUR CODE HERE #########

    ##################################
    
    # Convert inputs to cvxopt format
    P = matrix(P_numpy, tc='d')
    q = matrix(q_numpy, tc='d')
    G = matrix(G_numpy, tc='d')
    h = matrix(h_numpy, tc='d')
    A = matrix(A_numpy, tc='d')
    b = matrix(b_numpy, tc='d')

    # Solve the QP problem to find Lagrange multipliers alpha
    solution = solvers.qp(P, q, G, h, A, b)
    alpha = np.ravel(solution['x'])

    return alpha

In [ ]:
class HardMarginSVM:
    def __init__(self, kwargs=None):
        self.kwargs = kwargs
        self.support_vectors = None
        self.w = None
        self.b = None

    def kernel_function(self, x, y):
        return compute_kernel_matrix(x, y, self.kwargs)
        
    def fit(self, X, y):

        K = self.kernel_function(X, X)
        n_samples, n_features = X.shape

        # Solve the dual optimization problem to obtain the Lagrange multipliers
        self.alpha = find_alpha(X, y, K)

        # Select support vectors
        support_vector_indices = self.alpha > 1e-8 
        self.alpha = self.alpha[support_vector_indices]
        self.support_vectors = X[support_vector_indices]
        self.support_vector_labels = y[support_vector_indices]

        # Compute the bias term b
        self.b = None
        ######### YOUR CODE HERE #########
        

        ##################################    

    def predict(self, X):

        pred = None

        ######### YOUR CODE HERE #########


        ##################################

        return pred

    def plot_decision_boundary(self, X, y):
        plt.figure(figsize=(10, 6))
        plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', marker='o', s=30, edgecolors='k', label='Data Points')
        plt.scatter(self.support_vectors[:, 0], self.support_vectors[:, 1], s=100, linewidth=1, facecolors='none',
                    edgecolors='k', label='Support Vectors')

        # Plot decision boundary
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
        grid = np.c_[xx.ravel(), yy.ravel()]
        Z = self.predict(grid).reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.3, cmap='bwr')
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        plt.legend()
        plt.show()

        

In [ ]:
# Generate a linearly separable dataset with support vectors closer to the decision boundary
def generate_linear_data():
    from sklearn.datasets import make_classification
    X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, class_sep=1.0, random_state=42)
    y = np.where(y == 0, -1, 1)
    return X, y

# Generate a non-linearly separable dataset
def generate_nonlinear_data():
    from sklearn.datasets import make_circles
    X, y = make_circles(n_samples=100, factor=0.5, noise=0.1)
    y = np.where(y == 0, -1, 1)
    return X, y

X_linear, y_linear = generate_linear_data()
X_no, y_no = generate_nonlinear_data()

dict_linear = {'kernel_type': 'linear'}
dict_gaussian = {'kernel_type': 'gaussian', 'gamma': 1}
dict_polynomial = {'kernel_type': 'polynomial', 'c': 1, 'degree': 3}

svm_linear = HardMarginSVM(dict_linear)
svm_linear.fit(X_linear, y_linear)
print("Linear SVM results:")
svm_linear.plot_decision_boundary(X_linear, y_linear)

svm_linear.fit(X_no, y_no)
print("Non-linear SVM results:")
svm_linear.plot_decision_boundary(X_no, y_no)

svm_gaussian = HardMarginSVM(dict_polynomial)
svm_gaussian.fit(X_no, y_no)
print("Gaussian SVM results:")
svm_gaussian.plot_decision_boundary(X_no, y_no)

### Soft Margin SVM with `cvxopt` Library

For the Soft Margin SVM, we allow for some misclassification errors by introducing a regularization parameter $C$ that controls the trade-off between maximizing the margin and minimizing the classification error. This introduces slack variables, modifying the optimization problem to allow a degree of violation for points that cannot be perfectly separated.

The dual formulation for the Soft Margin SVM is as follows:

$$
\max_{\alpha} \; W(\alpha) = \sum_{i=1}^{m} \alpha_i - \frac{1}{2} \sum_{i,j=1}^{m} y^{(i)} y^{(j)} \alpha_i \alpha_j \langle x^{(i)}, x^{(j)} \rangle
$$

subject to:
$$
0 \leq \alpha_i \leq C, \quad i = 1, \dots, m
$$
and
$$
\sum_{i=1}^{m} \alpha_i y^{(i)} = 0
$$

### Standard Form for `cvxopt.solvers.qp`

The `cvxopt.solvers.qp` function solves problems in the following standard form:

$$
\begin{aligned}
&\min \frac{1}{2} x^T P x + q^T x \\
\text{subject to:} \quad & G x \leq h \\
& A x = b
\end{aligned}
$$

where:
- $x$ is the vector of variables we are solving for (in our case, the vector of Lagrange multipliers $\alpha$),
- $P$ is a matrix representing the quadratic coefficients of the objective function,
- $q$ is a vector representing the linear coefficients of the objective function,
- $G$ and $h$ represent inequality constraints,
- $A$ and $b$ represent equality constraints.

### Mapping the Soft Margin SVM Dual Problem to the QP Form

1. **Variables**:
   - Here, $x = \alpha$, the vector of Lagrange multipliers.

2. **Objective Function**:
   - The dual objective function can be written as:
     $$
     W(\alpha) = \sum_{i=1}^{m} \alpha_i - \frac{1}{2} \sum_{i,j=1}^{m} y^{(i)} y^{(j)} \alpha_i \alpha_j K_{ij}
     $$
     where $K_{ij} = \langle x^{(i)}, x^{(j)} \rangle$ is the Gram matrix (or kernel matrix).
   
   - Since `cvxopt` expects a minimization problem, we need to minimize $-W(\alpha)$, which becomes:
     $$
     \min \; -\sum_{i=1}^{m} \alpha_i + \frac{1}{2} \sum_{i,j=1}^{m} y^{(i)} y^{(j)} \alpha_i \alpha_j K_{ij}
     $$

   - This gives:
     $$
     \begin{align}
     &\text{Matrix } P: & P_{ij} &= y^{(i)} y^{(j)} K_{ij} \\
     &\text{Vector } q: & q_i &= -1 \quad \text{(since the linear term is } -\sum_{i=1}^{m} \alpha_i \text{)}
     \end{align}
     $$

3. **Constraints**:
   - Inequality Constraints: We have $0 \leq \alpha_i \leq C$, which translates to:
     $$
     G = \begin{bmatrix} -I \\ I \end{bmatrix}, \quad h = \begin{bmatrix} 0 \\ C \end{bmatrix}
     $$
     where $-I$ and $I$ represent the negative and positive identity matrices, respectively, ensuring $\alpha$ values remain within the interval $[0, C]$.

   - Equality Constraint $\left(\sum_{i=1}^{m} \alpha_i y^{(i)} = 0\right)$:
     $$
     A = y^T \quad \text{(labels vector)}, \quad b = 0
     $$


In [ ]:
def find_alpha_soft(X, y, K, C):

    n_samples, n_features = X.shape
    P_numpy, q_numpy, G_numpy, h_numpy, A_numpy, b_numpy = None, None, None, None, None, None

    # Convert inputs to cvxopt format
    ######### YOUR CODE HERE #########





    

    ##################################
    
    # Convert inputs to cvxopt format
    P = matrix(P_numpy, tc='d')
    q = matrix(q_numpy, tc='d')
    G = matrix(G_numpy, tc='d')
    h = matrix(h_numpy, tc='d')
    A = matrix(A_numpy, tc='d')
    b = matrix(b_numpy, tc='d')

    # Solve the QP problem to find Lagrange multipliers alpha
    solution = solvers.qp(P, q, G, h, A, b)
    alpha = np.ravel(solution['x'])

    return alpha

In [ ]:
import numpy as np
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt

class SoftMarginSVM:
    def __init__(self, kwargs=None):
        self.kwargs = kwargs
        self.C = kwargs['C']
        self.alpha = None
        self.support_vectors = None
        self.w = None
        self.b = None

    def kernel_function(self, x, y):
        return compute_kernel_matrix(x, y, self.kwargs)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Compute the Kernel matrix
        K = self.kernel_function(X, X)

        self.alpha = find_alpha_soft(X, y, K, self.C)

        # Select support vectors
        support_vector_indices = (self.alpha > 1e-8) & (self.alpha < self.kwargs.get('C', np.inf))
        self.alpha = self.alpha[support_vector_indices]
        self.support_vectors = X[support_vector_indices]
        self.support_vector_labels = y[support_vector_indices]

        # Compute the bias term b
        ######### YOUR CODE HERE #########

        
        ##################################

    def predict(self, X):

        pred = None 

        ######### YOUR CODE HERE #########


        ##################################

        return pred

    def plot_decision_boundary(self, X, y):
        plt.figure(figsize=(10, 6))
        plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', marker='o', s=30, edgecolors='k', label='Data Points')
        plt.scatter(self.support_vectors[:, 0], self.support_vectors[:, 1], s=100, linewidth=1, facecolors='none',
                    edgecolors='k', label='Support Vectors')

        # Plot decision boundary
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
        grid = np.c_[xx.ravel(), yy.ravel()]
        Z = self.predict(grid).reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.3, cmap='bwr')
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        plt.legend()
        plt.show()


In [ ]:
# Define a function to generate overlapping linear data
def generate_challenging_linear_data():
    # Generate a mostly separable dataset
    X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, 
                               n_clusters_per_class=1, class_sep=1.5, random_state=42)
    y = np.where(y == 0, -1, 1)

    # Introduce some overlap by adding noise
    np.random.seed(42)
    n_overlap = 5  # number of overlapping points
    overlap_indices = np.random.choice(range(50, 100), n_overlap, replace=False)
    
    # Flip the labels of the overlapping points to create misclassifications
    y[overlap_indices] = -y[overlap_indices]
    
    return X, y

# Generate the overlapping linear data
X_challenging, y_challenging = generate_challenging_linear_data()

# Plot the data to visualize the overlap
plt.figure(figsize=(8, 6))
plt.scatter(X_challenging[:, 0], X_challenging[:, 1], c=y_challenging, cmap='bwr', marker='o', s=50, edgecolors='k')
plt.title("Challenging Linear Data with Overlap")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

dict_linear = {'kernel_type': 'linear'}
dict_gaussian = {'kernel_type': 'gaussian', 'gamma': 1}
dict_polynomial = {'kernel_type': 'polynomial', 'c': 1, 'degree': 3}
# Try to fit a Hard Margin SVM (will likely fail)
try:
    
    hard_margin_svm = HardMarginSVM(dict_gaussian)
    hard_margin_svm.fit(X_challenging, y_challenging)
    print("Hard Margin SVM results:")
    hard_margin_svm.plot_decision_boundary(X_challenging, y_challenging)
except Exception as e:
    print("Hard Margin SVM failed due to non-separable data:", e)


dict_linear = {'kernel_type': 'linear', 'C': 0.5}
dict_gaussian = {'kernel_type': 'gaussian', 'gamma': 1, 'C': 1}
dict_polynomial = {'kernel_type': 'polynomial', 'c': 1, 'degree': 3, 'C': 1}
# Fit a Soft Margin SVM (should succeed with overlapping data)
soft_margin_svm = SoftMarginSVM(dict_linear)
soft_margin_svm.fit(X_challenging, y_challenging)
print("Soft Margin SVM results:")
soft_margin_svm.plot_decision_boundary(X_challenging, y_challenging)
